In [ ]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal

In [ ]:
dataframe = pd.read_excel('Türkçe Nefret Söylemi Veri Seti_1k.xlsx', sheet_name='1000 Tweet')

dataframe = dataframe.iloc[1:, [1, 2]]

In [ ]:
dataframe

In [ ]:
morphology = zp.TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

In [ ]:
resultDict = {'__tweet__': [], '__label__': []}

cancaled = False

def handler(signum, frame):
    global cancaled
    cancaled = True
signal.signal(signal.SIGINT, handler)

for index, row in tqdm(dataframe.iterrows()):
    if cancaled:
        break
    try:
        normalizedSentence = normalizer.normalize(row[0])
        analysis = morphology.analyze_sentence(normalizedSentence)
        after = morphology.disambiguate(normalizedSentence, analysis)
    except:
        print("Error: ", normalizedSentence)
        continue
    words = {}
    for sentence in after.best_analysis():
        if (words.get(sentence.get_stem())):
            words[sentence.get_stem()] += 1
        else:
            words[sentence.get_stem()] = 1
    
    columns = set(list(words.keys()) + list(resultDict.keys())[2:])
    for column in columns:
        if column not in resultDict.keys():
            resultDict[column] = [0] * len(resultDict['__tweet__'])
        if column in words:
            resultDict[column].append(int(words[column]))
        else:
            resultDict[column].append(int(0))
    resultDict['__tweet__'].append(str(row[0]).replace('\n', ' '))
    resultDict['__label__'].append(str(row[1]) if row[1] != None and type(row[1]) is str else "")

In [ ]:

list(resultDict.items())[1]


In [ ]:
resultDataFrame = pd.DataFrame(resultDict)

In [ ]:
resultDataFrame

In [ ]:
resultDataFrame.to_csv("result.csv")